In [2]:
import torch
import torch.nn as nn
import torch.optim as Adam
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

device="cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# data_df= pd.read_json('C:/Users/nihal/Downloads/archive (1)/Sarcasm_Headlines_Dataset.json', lines=True)
# data_df.dropna(inplace=True)
# data_df.drop_duplicates(inplace=True)
# data_df.drop(("article_link"), axis=1, inplace=True)
# print(f"Dataset shape: {data_df.shape}")
# print(data_df.head())


Dataset shape: (26708, 2)
                                            headline  is_sarcastic
0  former versace store clerk sues over secret 'b...             0
1  the 'roseanne' revival catches up to our thorn...             0
2  mom starting to fear son's web series closest ...             1
3  boehner just wants wife to listen, not come up...             1
4  j.k. rowling wishes snape happy birthday in th...             0


In [9]:
X_train,X_test,y_train,y_test= train_test_split(np.array(data_df['headline']), np.array(data_df['is_sarcastic']), test_size=0.3)
X_val,X_test,y_val,y_test= train_test_split(X_test, y_test, test_size=0.5)
print("Training Size: ", X_train.shape[0], "which is ", round(X_train.shape[0]/data_df.shape[0],4)*100, "% of total data")
print("Validation Size: ", X_val.shape[0], "which is ", round(X_val.shape[0]/data_df.shape[0],4)*100, "% of total data")
print("Testing Size: ", X_test.shape[0], "which is ", round(X_test.shape[0]/data_df.shape[0],4)*100, "% of total data")

Training Size:  18695 which is  70.0 % of total data
Validation Size:  4006 which is  15.0 % of total data
Testing Size:  4007 which is  15.0 % of total data


In [10]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
bert_model = AutoModel.from_pretrained("google-bert/bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\nihal\OneDrive\Desktop\python-codes\cudenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nihal\.cache\huggingface\hub\models--google-bert--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [11]:
class dataset(Dataset):
    def __init__(self,X,Y):
        self.X = [tokenizer(X,
                  max_length=100,
                  padding='max_length',
                  return_tensors='pt',).to(device) 
                  for X in X
        ]
        self.Y=torch.tensor(Y,dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.X)
    def __getitem__(self,idx):
        return self.X[idx], self.Y[idx]
    
train_dataset= dataset(X_train,y_train)
val_dataset= dataset(X_val,y_val)   
test_dataset= dataset(X_test,y_test)

In [12]:
BATCH_SIZE=32
EPOCHS=15
LR=0.001

In [13]:
train_dataloader= DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader= DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader= DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)